In [3]:
import sys
sys.path.append("..")

import numpy as np
import pandas as pd
from time import time
from tqdm import tqdm
from pathlib import Path
from src.data_utils import clip_and_scale_image

# Load model

In [4]:
import os
import torch
from pathlib import Path

from src.tilenet import make_tilenet

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
cuda = torch.cuda.is_available()

def load_model(model_filepath, bands = 13, z_dim = 512):
    
    model_dir = Path('/storage/tile2vec/models')
    model_fn = model_dir / model_filepath # specify which model weights are to be loaded
    
    bands = bands # number of bands in the input data - should matche the model
    z_dim = z_dim # output dimension of the last layer of the encoder - should match the model


    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    cuda = torch.cuda.is_available()


    tilenet = make_tilenet(in_channels=bands, z_dim=z_dim) 
    if cuda: 
        tilenet.cuda()
        
    print("Model filepath: ", model_fn)

    checkpoint = torch.load(model_fn)
    tilenet.load_state_dict(checkpoint)
    tilenet.eval()
    print("Model succesfully loaded")
    return tilenet

In [5]:
val_df = pd.read_csv("/storage/EuroSATallBands/validation.csv")
n_tiles = len(val_df)
tiles_path = Path("/storage/tile2vec/npy/val")

X_bare_val = np.zeros((n_tiles, 64 * 64 * 13), dtype=np.float32)
X_norm_val = np.zeros((n_tiles, 64 * 64 * 13), dtype=np.float32)

for index in tqdm(range(n_tiles)):
    tile = np.load(tiles_path / f"{index}.npy")
    X_norm_val[index] = clip_and_scale_image(tile, img_type="landsat").flatten()
    X_bare_val[index] = tile.flatten()
    
y_val = val_df["Label"]

test_df = pd.read_csv("/storage/EuroSATallBands/test.csv")
n_tiles = len(test_df)
tiles_path = Path("/storage/tile2vec/npy/test")

X_bare_test = np.zeros((n_tiles, 64 * 64 * 13), dtype=np.float32)
X_norm_test = np.zeros((n_tiles, 64 * 64 * 13), dtype=np.float32)

for index in tqdm(range(n_tiles)):
    tile = np.load(tiles_path / f"{index}.npy")
    X_norm_test[index] = clip_and_scale_image(tile, img_type="landsat").flatten()
    X_bare_test[index] = tile.flatten()
    
y_test = test_df["Label"]


train_df = pd.read_csv("/storage/EuroSATallBands/train.csv")
n_tiles = len(train_df)
tiles_path = Path("/storage/tile2vec/npy/train")

X_bare_train = np.zeros((n_tiles, 64 * 64 * 13), dtype=np.float32)
X_norm_train = np.zeros((n_tiles, 64 * 64 * 13), dtype=np.float32)

for index in tqdm(range(n_tiles)):
    tile = np.load(tiles_path / f"{index}.npy")
    X_norm_train[index] =  clip_and_scale_image(tile, img_type="sentinel").flatten()

    X_bare_train[index] = tile.flatten()
    
y_train = train_df["Label"]

100%|██████████| 19317/19317 [00:11<00:00, 1727.10it/s]


In [6]:
y_train = train_df["Label"]
y_validation = val_df["Label"]
y_test = test_df["Label"]

In [7]:
X_norm_valtest = np.concatenate((X_norm_val, X_norm_test), axis=0)
y_valtest = np.concatenate((y_validation, y_test), axis=0)

We also want to compare our model with different methods of dimentionality reduction. Therefore we create variables to evaluate performance of different methods. In the code above we create 6 matrices, two for each part of our data. Each matrix contains observations from sets. For instance, X_*_train contains images as rows, which are flattened to a row-vector. Each collum contains information about pixel on a specific position in the image.

## Embeddings
Below there is a function that creates embeddings from the provided path and the model.

In [8]:

def create_embeddings_tile2vec(tilenet, 
                               df_filepath: str | Path, 
                               tiles_path: str | Path, 
                               img_type: str = "sentinel", 
                               bands: int = 13, 
                               z_dim: int = 512):
    """
    function creates matrix X and y containing embeddings and labels, loads the tiles from directory `tiles_path`
    """
    df_filepath = Path(df_filepath)
    tiles_path = Path(tiles_path)
    df = pd.read_csv(df_filepath)
    n_tiles = len(df)    
    
    X = np.zeros((n_tiles, z_dim))
    
    t0 = time()
    # this solution to iterate over examples is very suboptimal, one should use torch dataset
    for index in tqdm(range(n_tiles)):
        # read the tile from provided filepath
        
        tile = np.load(tiles_path / f"{index}.npy")  
        tile = clip_and_scale_image(tile, img_type=img_type)[:, :bands,:, :]
        tile = torch.from_numpy(tile).float()
        tile = (tile)
        if cuda: 
            tile = tile.cuda()
        z = tilenet.encode(tile)
        if cuda: 
            z = z.cpu()
        z = z.data.numpy()
        
        X[index,:] = z

    t1 = time()
    print('Embedded {} tiles: {:0.3f}s'.format(n_tiles, t1-t0))
    
    y = df['Label'].values
    
    return X, y

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

rf = RandomForestClassifier()

In [10]:
def compare_results(X, y, model, folds = 5, model_name = ""):
    # simple method to evaluate performance of model using cross validation
    if model_name == "":
        model_name = model.__class__.__name__
    
    scores = cross_val_score(rf, X, y, cv=folds)
    print("Averaged accuracy for model {}: {:.2f}±{:.2f}%".format(model_name, scores.mean()*100, scores.std()*100))

# Evaluation
Finally we perform tests, each section contains code and results of evaluation - the results and model or method type

## Default model scaling
Here we are going to check the performance of model with no band-specfic scaling

In [11]:
no_clipping = load_model("TileNet_Distant_Diff.ckpt")
X_validation, y_validation = create_embeddings_tile2vec(no_clipping, '/storage/EuroSATallBands/validation.csv', '/storage/tile2vec/tif/val', "landsat")
X_test, y_test = create_embeddings_tile2vec(no_clipping, '/storage/EuroSATallBands/test.csv', '/storage/tile2vec/tif/test', "landsat")

Model filepath:  /storage/tile2vec/models/TileNet_Distant_Diff.ckpt
Model succesfully loaded


100%|██████████| 5519/5519 [00:19<00:00, 290.06it/s]


Embedded 5519 tiles: 19.029s


100%|██████████| 2759/2759 [00:09<00:00, 306.48it/s]

Embedded 2759 tiles: 9.004s


In [12]:
X = np.concatenate((X_validation, X_test), axis=0)  
y = np.concatenate((y_validation, y_test), axis=0)

In [13]:
compare_results(X, y, rf, folds = 5)

Averaged accuracy for model RandomForestClassifier: 65.92±0.71%


In [14]:
rf.fit(X, y)
print(rf.score(X, y))
confusion_matrix(y, rf.predict(X))

1.0


array([[ 900,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,  900,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,  900,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,  750,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,  750,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,  600,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,  750,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,  900,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,  750,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0, 1078]])

In [15]:
cross_val_score(rf, X, y, cv=5)

array([0.67270531, 0.65700483, 0.65700483, 0.65317221, 0.6652568 ])

In [1]:
X_train, y_train = create_embeddings_tile2vec(no_clipping,
                                    '/storage/EuroSATallBands/train.csv',
                                    '/storage/tile2vec/npy/train',
                                    "landsat")

NameError: name 'create_embeddings_tile2vec' is not defined

In [2]:
compare_results(X_train, y_train, rf, folds = 5)

NameError: name 'compare_results' is not defined

array([0.59842995, 0.58454106, 0.58695652, 0.58610272, 0.58670695])

In [56]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

cross_val_score(rf,X,y,cv=5)

array([0.59480676, 0.59057971, 0.58333333, 0.58670695, 0.58912387])

In [62]:
cross_val_score(rf, X_train, y_train, cv=5)

array([0.60662526, 0.60300207, 0.60781776, 0.61584261, 0.60859436])

In [65]:
rf.fit(X_train, y_train)
print(rf.score(X_train,y_train))
confusion_matrix(y_train,rf.predict(X_train))

1.0


array([[2100,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0, 2100,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0, 2100,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0, 1750,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0, 1750,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0, 1400,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0, 1750,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0, 2100,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0, 1750,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0, 2517]])